## Packages

From termainal I did:

```
pipenv install
pipenv install jupyter pymysql sqlalchemy requests
```


In [81]:
import pymysql.cursors
import requests
from datetime import datetime
from sqlalchemy import create_engine
import hashlib 
import os
import json
import time
from collections import defaultdict
from flask import jsonify
import re

# variables we'll need
host = os.environ['DBHOST']
port = 3306
dbname = "collab"
user = os.environ['DBUSER']
password = os.environ['DBPASSWORD']

In [90]:
def checkId(goog_data):
    
    # grab the last bit of the session ID to use as default
    # this regex does that somehow! :-)
    session = re.search(r'([^-]+$)', goog_data['session']).group(1)
    
    # no source, so return the session ID
    if 'source' not in goog_data['originalDetectIntentRequest'].keys():
        return f'test-session-{session}'

    
    # for twilio, it's the From phone number
    if goog_data['originalDetectIntentRequest']['source'] == "twilio":
        return goog_data['originalDetectIntentRequest']['payload']['data']['From']
    
    # otherwise we have an unknown platform
    return f'unknown platform {session}'

In [97]:
def update_first_contact(connection, now, identifier):
        
    ## Add a new item to the first_contact, but chill 
    ## if it already exists

    sql = "INSERT IGNORE INTO first_contact (`created_at`, `identifier`) VALUES (%s, %s)"

    with connection.cursor() as cursor:
            # Create a new record
            cursor.execute(sql, ( now, identifier ))

    connection.commit()
    
    return True

In [98]:
def insert_into_log(connection, now, identifier, key, value):
    
    # Create a new record
    sql = "INSERT INTO `log` (`created_at`, `identifier`, `item_key`, `item_value`) VALUES (%s, %s, %s, %s)"

    with connection.cursor() as cursor:
            # Create a new record
            cursor.execute(sql, ( now, identifier, key, value ))

    connection.commit()
    
    return True

In [99]:
def update_column_tracker(connection, now, column):

    # Create a new record
    sql = "INSERT IGNORE INTO `column_tracker` (`created_at`, `col`) VALUES (%s, %s)"

    with connection.cursor() as cursor:
            # Create a new record
            cursor.execute(sql, ( now, column ))

    connection.commit()
    
    return True

In [100]:
#### Main function ####

def handler(incoming):
    
    # incoming data will be structured as described
    # here: https:#cloud.google.com/dialogflow/docs/fulfillment-webhook#webhook_request
    
    #PROD from_google = incoming.json
    from_google = incoming ### for testing only
    
    # Open the database connection
    connection = pymysql.connect(host=host,
                                 user=user,
                                 password=password,
                                 db=dbname,
                                 charset='utf8mb4',
                                 cursorclass=pymysql.cursors.DictCursor)
    
    identifier = checkId(from_google)
    user_said = from_google['queryResult']['queryText']
    now = datetime.now().isoformat(sep=' ', timespec='seconds') # => '2020-05-12 18:54:54'
    
    # Update the first_contact
    update_first_contact(connection, now, identifier)
    
    ## Add the intent as a column and a log item
    intent = from_google['queryResult']['intent']['displayName']
    update_column_tracker(connection, now, intent)
    insert_into_log(connection, now, identifier, intent, user_said) 
    
    ## Add every parameter as a column & log item
    parameters = from_google['queryResult']['parameters']
    
    for key in parameters.keys():
        update_column_tracker(connection, now, key)
        insert_into_log(connection, now, identifier, key, parameters[key])
    
    ## Add the raw text into the log
    insert_into_log(connection, now, identifier, "raw_text", user_said) 
    
    # close the database
    connection.close()
    
    # send back to google the messages already established there
    to_google = {
        "fulfillmentMessages": from_google['queryResult']['fulfillmentMessages']
    }
    
    #PROD return jsonify(to_google)
    return (to_google)

In [101]:
test_payload = {
    "responseId": "a96d4f9c-7b85-4a04-8b43-be609c8c695e-0f0e27e1",
    "queryResult": {
        "queryText": "Hello",
        "action": "input.welcome",
        "parameters": {},
        "allRequiredParamsPresent": True,
        "fulfillmentText": "Hello! What city are you in now?",
        "fulfillmentMessages": [
            {
                "text": {
                    "text": [
                        "Hello! What city are you in now?"
                    ]
                }
            }
        ],
        "outputContexts": [
            {
                "name": "projects/propublicafruittest-pyrata/agent/sessions/96c6d41c-b1b0-e1f8-4ca7-7e9945fa1a0a/contexts/waiting-city",
                "lifespanCount": 5
            },
            {
                "name": "projects/propublicafruittest-pyrata/agent/sessions/96c6d41c-b1b0-e1f8-4ca7-7e9945fa1a0a/contexts/__system_counters__",
                "parameters": {
                    "no-input": 0,
                    "no-match": 0
                }
            }
        ],
        "intent": {
            "name": "projects/propublicafruittest-pyrata/agent/intents/10d4a02d-ac4c-4669-8782-8beffec91447",
            "displayName": "Default Welcome Intent"
        },
        "intentDetectionConfidence": 1,
        "languageCode": "en"
    },
    "originalDetectIntentRequest": {
        "payload": {}
    },
    "session": "projects/propublicafruittest-pyrata/agent/sessions/96c6d41c-b1b0-e1f8-4ca7-7e9945fa1a0a"
}

In [102]:
## Trying it
handler(test_payload)

{'fulfillmentMessages': [{'text': {'text': ['Hello! What city are you in now?']}}]}